# Feature Engineering

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

DATASET = Path("datasets/lotto/data_processed.csv")

In [2]:
# Read into data frame
dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"', parse_dates=['DrawDate'], dtype={'PrizeType': str})

In [3]:
df = dataset.copy()

In [4]:
df.columns

Index(['DrawNo', 'DrawDate', 'PrizeType', 'LuckyNo', 'DaysSinceLastExactMatch',
       'DaysUntilNextExactMatch', 'DaysSinceLastAnyMatch',
       'DaysUntilNextAnyMatch', '1st_digit', '2nd_digit',
       ...
       'pos_4_0_freq', 'pos_4_1_freq', 'pos_4_2_freq', 'pos_4_3_freq',
       'pos_4_4_freq', 'pos_4_5_freq', 'pos_4_6_freq', 'pos_4_7_freq',
       'pos_4_8_freq', 'pos_4_9_freq'],
      dtype='object', length=115)

### Matrix Factorization